<a href="https://colab.research.google.com/github/hmatny/style_translation/blob/master/TPU_nmt_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License").

# Neural Machine Translation with Attention

<table class="tfo-notebook-buttons" align="left"><td>
<a target="_blank"  href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/nmt_with_attention/nmt_with_attention.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank"  href="https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/eager/python/examples/nmt_with_attention/nmt_with_attention.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

This notebook trains a sequence to sequence (seq2seq) model for Spanish to English translation using [tf.keras](https://www.tensorflow.org/programmers_guide/keras) and [eager execution](https://www.tensorflow.org/programmers_guide/eager). This is an advanced example that assumes some knowledge of sequence to sequence models.

After training the model in this notebook, you will be able to input a Spanish sentence, such as *"¿todavia estan en casa?"*, and return the English translation: *"are you still at home?"*

The translation quality is reasonable for a toy example, but the generated attention plot is perhaps more interesting. This shows which parts of the input sentence has the model's attention while translating:

<img src="https://tensorflow.org/images/spanish-english.png" alt="spanish-english attention plot">

Note: This example takes approximately 10 mintues to run on a single P100 GPU.

In [0]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

#tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time

print(tf.__version__)

1.13.1


In [0]:
import json
import os
import pprint
import re
import time

use_tpu = False #@param {type:"boolean"}
bucket = 'cs378_bert' #@param {type:"string"}

# assert bucket, 'Must specify an existing GCS bucket name'
# print('Using bucket: {}'.format(bucket))

if use_tpu:
    assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

MODEL_DIR = 'gs://{}/{}'.format(bucket, time.strftime('shakespeare'))
print('Using model dir: {}'.format(MODEL_DIR))

from google.colab import auth
auth.authenticate_user()


if 'COLAB_TPU_ADDR' in os.environ:
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_ADDRESS)

  # Upload credentials to TPU.
  with tf.Session(TPU_ADDRESS) as sess:    
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

else:
  TPU_ADDRESS=''

with tf.Session(TPU_ADDRESS) as session:
  pprint.pprint(session.list_devices())

Using model dir: gs://cs378_bert/shakespeare

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

TPU address is grpc://10.78.198.10:8470
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5803830991548136265),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 10897063943721619062),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13679249626147221069),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 8115314328399049858),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 7484135093204464864),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 14200427662848957653),
 _DeviceAttributes(/job:tpu_worker/replic

## Download and prepare the dataset

We'll use a language dataset provided by http://www.manythings.org/anki/. This dataset contains language translation pairs in the format:

```
May I borrow this book?	¿Puedo tomar prestado este libro?
```

There are a variety of languages available, but we'll use the English-Spanish dataset. For convenience, we've hosted a copy of this dataset on Google Cloud, but you can also download your own copy. After downloading the dataset, here are the steps we'll take to prepare the data:

1. Add a *start* and *end* token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.

In [0]:
DATA_LINKS = ["/content/sparknotes/", "/content/enotes/"]
 
MODERN_FILENAME   = "all_modern.snt.aligned"
ORIGINAL_FILENAME = "all_original.snt.aligned"

TRAIN_SUFFIX = "_train"
DEV_SUFFIX   = "_dev"

CACHE_DIR = "/content/cache/"

MODERN_PATH   = CACHE_DIR + MODERN_FILENAME
ORIGINAL_PATH = CACHE_DIR + ORIGINAL_FILENAME

MODERN_TRAIN_PATH   = MODERN_PATH +   TRAIN_SUFFIX
MODERN_DEV_PATH     = MODERN_PATH +   DEV_SUFFIX
ORIGINAL_TRAIN_PATH = ORIGINAL_PATH + TRAIN_SUFFIX
ORIGINAL_DEV_PATH   = ORIGINAL_PATH + DEV_SUFFIX

ROOT_DIR = os.getcwd()

RANDOM_SEED=42

def get_dir(relative_path=''):
    return os.path.join(ROOT_DIR, relative_path)

In [0]:
!git clone https://github.com/cocoxu/Shakespeare.git
!cp -r ./Shakespeare/data/align/plays/merged/ ./sparknotes/
!cp -r ./Shakespeare/data/align/plays2/merged/ ./enotes/
!cd ./sparknotes/ && ls && pwd
!cd ./enotes/ && ls && pwd
!mkdir /content/cache/

Cloning into 'Shakespeare'...
remote: Enumerating objects: 9016, done.
remote: Total 9016 (delta 0), reused 0 (delta 0), pack-reused 9016
Receiving objects: 100% (9016/9016), 556.83 MiB | 34.54 MiB/s, done.
Resolving deltas: 100% (3354/3354), done.
Checking out files: 100% (4160/4160), done.
antony-and-cleopatra_modern.snt.aligned    merchant_original.snt.aligned
antony-and-cleopatra_original.snt.aligned  msnd_modern.snt.aligned
asyoulikeit_modern.snt.aligned		   msnd_original.snt.aligned
asyoulikeit_original.snt.aligned	   muchado_modern.snt.aligned
errors_modern.snt.aligned		   muchado_original.snt.aligned
errors_original.snt.aligned		   othello_modern.snt.aligned
hamlet_modern.snt.aligned		   othello_original.snt.aligned
hamlet_original.snt.aligned		   richardiii_modern.snt.aligned
henryv_modern.snt.aligned		   richardiii_original.snt.aligned
henryv_original.snt.aligned		   romeojuliet_modern.snt.aligned
juliuscaesar_modern.snt.aligned		   romeojuliet_original.snt.aligned
juliuscaes

In [0]:
path_to_file ='/content/cache/all_pairs'

def get_shakespeare_parallel_set():    
    
  all_pairs_file = open(path_to_file,'w')

  modern = []
  original = []

  for aligned_data in DATA_LINKS:
    for root, dirs, filenames in os.walk(get_dir(aligned_data)):
      for filename in sorted(filenames):
        with open(os.path.join(get_dir(aligned_data), filename), 'r') as f:
          for line in f:
            if '_modern.snt.aligned' in filename:
              modern.append(line.strip())
            elif '_original.snt.aligned' in filename:
              original.append(line.strip())
            else:
                pass
  for modern, original in zip(modern, original):
    all_pairs_file.write(modern + "\t" + original + "\n")
  all_pairs_file.close()
  !wc -l '/content/cache/all_pairs'
  f = open('/content/cache/all_pairs')
  for i in range(5):
    print(f.readline())
  f.close()
get_shakespeare_parallel_set()

31444 /content/cache/all_pairs
I have half a mind to hit you before you speak again.	I have a mind to strike thee ere thou speak’st.

But if Antony is alive, healthy, friendly with Caesar, and not Caesar’s prisoner, I’ll shower you with gold and pearls.	Yet if thou say Antony lives, is well, Or friends with Caesar, or not captive to him, I’ll set thee in a shower of gold and hail Rich pearls upon thee.

Madam, he’s well.	Madam, he’s well.

That’s well spoken.	Well said.

And he’s friendly with Caesar.	And friends with Caesar.



In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [0]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    
    return word_pairs

In [0]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))
    
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1
    
    for word, index in self.word2idx.items():
      self.idx2word[index] = word

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)


def load_dataset(path, num_examples):
    # creating cleaned input, output pairs
    pairs = create_dataset(path, num_examples)

    # index language using the class defined above    
    inp_lang = LanguageIndex(sp for en, sp in pairs)
    targ_lang = LanguageIndex(en for en, sp in pairs)
    
    # Vectorize the input and target languages
    
    # Spanish sentences
    input_tensor = [[inp_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]
    
    # English sentences
    target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

### Limit the size of the dataset to experiment faster (optional)

Training on the complete dataset of >100,000 sentences will take a long time. To train faster, we can limit the size of the dataset to 30,000 sentences (of course, translation quality degrades with less data):

In [0]:
# Try experimenting with the size of that dataset
num_examples = 3000
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file, num_examples)

In [0]:
# # Creating training and validation sets using an 80-20 split
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# # Show length
# len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

### Create a tf.data dataset

In [0]:
BATCH_SIZE = 32
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

# orig_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
# dataset = orig_dataset.batch(BATCH_SIZE, drop_remainder=True)
# print(dataset)

# def map_fn(inp, targ):
# #   print(offset, x)
# #   inp, targ = x
# #   print(inp, targ)
#   return {"source": inp, "target":targ}

# inp_dataset = tf.data.Dataset.from_tensor_slices(input_tensor_train)
# targ_dataset = tf.data.Dataset.from_tensor_slices(target_tensor_train)
# data_mapped = dataset.map(map_fn)
# print(data_mapped)

In [0]:
BATCH_SIZE = 32

def input_fn(params):
  # Try experimenting with the size of that dataset
  num_examples = 3000
  input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file, num_examples)

  # Creating training and validation sets using an 80-20 split
  input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

  # Show length
  print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

  BUFFER_SIZE = len(input_tensor_train)
  N_BATCH = BUFFER_SIZE//BATCH_SIZE
  print("nbatch", N_BATCH)
  embedding_dim = 256
  units = 1024
  vocab_inp_size = len(inp_lang.word2idx)
  vocab_tar_size = len(targ_lang.word2idx)

  orig_dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
  
  def map_fn(inp, targ):
    return {"source": inp, "target":targ}

  data_mapped = orig_dataset.map(map_fn)
  print("data_mapped before batch", data_mapped)
  data_mapped = data_mapped.repeat()
  data_mapped = data_mapped.batch(BATCH_SIZE, drop_remainder=True)
  #data_mapped = data_mapped.apply(tf.contrib.data.batch_and_drop_remainder(batch_size))
  print("data_mapped after batch", data_mapped)
  return data_mapped.prefetch(2)


## Write the encoder and decoder model

Here, we'll implement an encoder-decoder model with attention which you can read about in the TensorFlow [Neural Machine Translation (seq2seq) tutorial](https://github.com/tensorflow/nmt). This example uses a more recent set of APIs. This notebook implements the [attention equations](https://github.com/tensorflow/nmt#background-on-the-attention-mechanism) from the seq2seq tutorial. The following diagram shows that each input word is assigned a weight by the attention mechanism which is then used by the decoder to predict the next word in the sentence.

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

The input is put through an encoder model which gives us the encoder output of shape *(batch_size, max_length, hidden_size)* and the encoder hidden state of shape *(batch_size, hidden_size)*. 

Here are the equations that are implemented:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

We're using *Bahdanau attention*. Lets decide on notation before writing the simplified form:

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

And the pseudo-code:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, 1)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
* `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
* `embedding output` = The input to the decoder X is passed through an embedding layer.
* `merged vector = concat(embedding output, context vector)`
* This merged vector is then given to the GRU
  
The shapes of all the vectors at each step have been specified in the comments in the code:

In [0]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    print("gpu")
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
  else:
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        #print("enc_output shape", enc_output.shape) # (4, 104, 1024)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [0]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [0]:
encoder

## Define the optimizer and the loss function

In [0]:
def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)

In [0]:
def train_fn(inp, targ, hidden=None):
  
  print("inp.shape", inp.shape)
  print("targ.shape", targ.shape)
  batch_size = inp.shape[0]

  encoder = Encoder(vocab_inp_size, embedding_dim, units, batch_size)
  decoder = Decoder(vocab_tar_size, embedding_dim, units, batch_size)
  
  enc_output, enc_hidden = encoder(inp, hidden)
  dec_hidden = enc_hidden

  dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * batch_size, 1)

# enc_output.shape (4, 112, 1024)
# enc_hidden.shape (4, 1024)
# dec_input.shape (32, 1)
  print("enc_output.shape", enc_output.shape)
  print("enc_hidden.shape", enc_hidden.shape)
  print("dec_input.shape", dec_input.shape)
  loss = 0
  # Teacher forcing - feeding the target as the next input
  for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_output)

      this_loss = loss_function(targ[:, t], predictions)
      print(targ[:, t], predictions)
      
      loss += this_loss
      print(t, this_loss)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)
      
  batch_loss = (loss / int(targ.shape[1]))
      
  optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
  if TPU_ADDRESS:
    optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)
  train_op = optimizer.minimize(loss, tf.train.get_global_step())
  
  #return loss, predictions, dec_hidden, attention_weights
  return tf.contrib.tpu.TPUEstimatorSpec(
      mode=tf.estimator.ModeKeys.TRAIN,
      loss=loss,
      train_op=train_op
  )

In [0]:
def eval_fn(inp, target):
  return None

In [0]:
def predict_fn(inp):
# def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
     

    encoder = Encoder(vocab_inp_size, embedding_dim, units, 1)
    decoder = Decoder(vocab_tar_size, embedding_dim, units, 1)

    result, sentence, attention_plot = evaluate(inp, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
     
    return tf.contrib.tpu.TPUEstimatorSpec(
      mode=tf.estimator.ModeKeys.PREDICT,
      predictions={'predictions': result}
    )
    #print('Input: {}'.format(sentence))
    #print('Predicted translation: {}'.format(result))
    
    #attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    #plot_attention(attention_plot, sentence.split(' '), result.split(' '))
    
    

In [0]:
def model_fn(features, labels, mode, params):
  if mode == tf.estimator.ModeKeys.TRAIN:
    return train_fn(features['source'], features['target'])
  if mode == tf.estimator.ModeKeys.EVAL:
    return eval_fn(features['source'], features['target'])
  if mode == tf.estimator.ModeKeys.PREDICT:
    return predict_fn(features['source'])

In [0]:
# def input_fn(params):
#   #inp, targ = orig_dataset
#   #print(orig_dataset)
#   return data_mapped.prefetch(2)

tf.reset_default_graph()
tf.set_random_seed(0)
with tf.Session() as session:
  #tf.compat.v1.disable_eager_execution()
  ds = input_fn({})
  #features = ds
  features = session.run(ds.make_one_shot_iterator().get_next())
  print(features['source'])
  print(features['target'])
  print(features['source'].shape)
  print(tf.data.experimental.cardinality(ds))

2400 2400 600 600
nbatch 75
data_mapped before batch <DatasetV1Adapter shapes: {source: (104,), target: (67,)}, types: {source: tf.int32, target: tf.int32}>
data_mapped after batch <DatasetV1Adapter shapes: {source: (32, 104), target: (32, 67)}, types: {source: tf.int32, target: tf.int32}>
[[   5 2373    2 ...    0    0    0]
 [   5 2325 2252 ...    0    0    0]
 [   5  441 2123 ...    0    0    0]
 ...
 [   5 3875    2 ...    0    0    0]
 [   5  635    2 ...    0    0    0]
 [   5 2286    2 ...    0    0    0]]
[[   5 2084    2 ...    0    0    0]
 [   5 3058 3307 ...    0    0    0]
 [   5  393 1874 ...    0    0    0]
 ...
 [   5 3326    2 ...    0    0    0]
 [   5 2671 1874 ...    0    0    0]
 [   5 3466 2490 ...    0    0    0]]
(32, 104)
Tensor("ExperimentalDatasetCardinality:0", shape=(), dtype=int64)


## Checkpoints (Object-based saving)

In [0]:
#MODEL_DIR = './training_checkpoints'
# checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# checkpoint = tf.train.Checkpoint(optimizer=optimizer,
#                                  encoder=encoder,
#                                  decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [0]:
def _make_estimator(num_shards, use_tpu=True):
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
  config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=MODEL_DIR,
    save_checkpoints_steps=100,
    tpu_config=tf.contrib.tpu.TPUConfig(
      num_shards=num_shards, iterations_per_loop=100)
  )
  
  estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=use_tpu,
    model_fn=model_fn,
    config=config,
    train_batch_size=BATCH_SIZE,
    eval_batch_size=1,
    predict_batch_size=1,
    params={}
  )
  
  return estimator


start = time.time()
with tf.Graph().as_default():
  estimator = _make_estimator(8, use_tpu)
  estimator.train(input_fn=input_fn, max_steps=400)

print('Time taken for 100 steps {} sec\n'.format(time.time() - start))

INFO:tensorflow:Using config: {'_model_dir': 'gs://cs378_bert/shakespeare', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.78.198.10:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f75a8d549b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.78.198.10:8470', '_evaluation_master': 'grpc://10.78.198.10:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=8, num_cores_per_replica

In [0]:
# EPOCHS = 10

# for epoch in range(EPOCHS):
#     start = time.time()
    
#     hidden = encoder.initialize_hidden_state()
#     total_loss = 0
    
#     for (batch, (inp, targ)) in enumerate(dataset):
#         loss = 0
        
#         with tf.GradientTape() as tape:
#             enc_output, enc_hidden = encoder(inp, hidden)
            
#             dec_hidden = enc_hidden
            
#             dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
#             # Teacher forcing - feeding the target as the next input
#             for t in range(1, targ.shape[1]):
#                 # passing enc_output to the decoder
#                 predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
#                 loss += loss_function(targ[:, t], predictions)
                
#                 # using teacher forcing
#                 dec_input = tf.expand_dims(targ[:, t], 1)
        
#         batch_loss = (loss / int(targ.shape[1]))
        
#         total_loss += batch_loss
        
#         variables = encoder.variables + decoder.variables
        
#         gradients = tape.gradient(loss, variables)
        
#         optimizer.apply_gradients(zip(gradients, variables))
        
#         if batch % 100 == 0:
#             print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
#                                                          batch,
#                                                          batch_loss.numpy()))
#     # saving (checkpoint) the model every 2 epochs
#     if (epoch + 1) % 2 == 0:
#       checkpoint.save(file_prefix = checkpoint_prefix)
    
#     print('Epoch {} Loss {:.4f}'.format(epoch + 1,
#                                         total_loss / N_BATCH))
#     print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [0]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [0]:
def _seed_input_fn(params):
  del params
  seed_txt = 'i am the king'
  input_tensor = [[inp_lang.word2idx[s] for s in seed_txt.split(' ')]]
#   # Padding the input and output tensor to the maximum length
#   input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
#                                                                maxlen=max_length_inp,
#                                                                padding='post')

  return tf.data.Dataset.from_tensors({'source': input_tensor})

In [0]:
next(estimator.predict(input_fn=_seed_input_fn))['predictions']

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:Error recorded from prediction_loop: 'Tensor' object has no attribute 'lower'
INFO:tensorflow:prediction_loop marked as finished


AttributeError: ignored

In [0]:
https://www.tensorflow.org/guide/using_tpu# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')
    
    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [0]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))
    
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence.split(' '), result.split(' '))

## Restore the latest checkpoint and test

In [0]:
from google.colab import auth
auth.authenticate_user()

!gsutil cp ./training_checkpoints/* gs://cs378_bert/nmt_model/outputs/

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
translate(u'I did not say that', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

In [0]:
translate(u'you are a cow', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

In [0]:
translate(u'I am angry', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

In [0]:
translate(u'you made me angry.', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

In [0]:
#translate(u'esta es mi vida.', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

In [0]:
#translate(u'todavia estan en casa?', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

In [0]:
# wrong translation
#translate(u'trata de averiguarlo.', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs
